In [ ]:
import os
import cv2

MODEL            = 'haarcascade_frontalface_default.xml'
TRAGET_FILES     = ['003.jpg', 'real_00001.jpg', 'real_00003.jpg', 'real_00004.jpg', 'real_00008.jpg', 'real_00018.jpg', 'real_00025.jpg', '003.jpg']
CHECK_FILES      = ['real_00004.jpg', 'real_00008.jpg']
TARGET_FILE_PATH = 'OK'
FACTOR           = 2.0
THICK            = 3
RED              = (0, 0, 255)
OUTPUT_PREFIX    = 'result_'

def detect_write(cascade, fname, scale = FACTOR) :
    """ １つの画像の顔検出を行う
    Arg:
        cas   : cv2.CascadeClassifier
        fname : str : ファイル名
    """
    fpath    = os.path.join(TARGET_FILE_PATH, fname)
    img      = cv2.imread(fpath)
    gray     = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces    = cascade.detectMultiScale(gray, scaleFactor = scale) # scaleFactorを調べたい
    
    for (x, y, w, h) in faces :
        cv2.rectangle(img, (x, y), (x + w, y + h),  color = RED, thickness = THICK)
        
    #output_file = OUTPUT_PREFIX + fname
    #output_path = os.path.join(TARGET_FILE_PATH, output_file)
    #ret = cv2.imwrite(output_path, img)
    return faces

def findout_all_files(target = TRAGET_FILES, scale = FACTOR) :
    cascade = cv2.CascadeClassifier(MODEL)
    for fn in target :  # test data (TARGET_FILES) loop
        faces = detect_write(cascade, fn, scale = scale)
    return

findout_all_files(target = TRAGET_FILES, scale = 2.6) # チェック対象のみ処理する scale = 2.0 がベスト?
